In [ ]:
!wget -qq https://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip
!unzip -qq balloon_dataset.zip

import os
import json
import cv2
import skimage.draw
import tensorflow as tf
import numpy as np

def parse(path, target_size = (512, 512), method = cv2.INTER_CUBIC):
    dir_path = os.path.dirname(path)
    with open(path, "r") as file:
        anno = json.loads(file.read())
    anno = [a for a in list(anno.values()) if a["regions"]]

    images = []
    masks = []
    for a in anno:
        if type(a['regions']) is dict:
            polygons = [r['shape_attributes'] for r in a['regions'].values()]
        else:
            polygons = [r['shape_attributes'] for r in a['regions']]
        image = cv2.cvtColor(cv2.imread(os.path.join(dir_path, a["filename"]), -1), cv2.COLOR_BGR2RGB)
        h, w = np.shape(image)[:2]
        mask = np.zeros((h, w, len(polygons)), dtype = np.uint8)
        for i, p in enumerate(polygons):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            rr = np.clip(rr, 0, h - 1)
            cc = np.clip(cc, 0, w - 1)
            mask[rr, cc, i] = 1
        
        if target_size is not None:
            image = cv2.resize(image, target_size[::-1], interpolation = method)
            mask = cv2.resize(mask, target_size[::-1], interpolation = method)
            if np.ndim(mask) == 2:
                mask = np.expand_dims(mask, axis = -1)
            mask = np.where(mask != 0, 1, 0)

        images.append(image)
        masks.append(np.transpose(mask, [2, 0, 1]))
    return images, masks

def pipe(data, batch_size = 2, shuffle = False):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    if shuffle:
        dataset = dataset.shuffle(buffer_size = batch_size * 10)
    dataset = dataset.batch(batch_size)
    #dataset = dataset.prefetch((batch_size * 2) + 1)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

tr_anno = "./balloon/train/via_region_data.json"
te_anno = "./balloon/val/via_region_data.json"

tr_x, tr_mask = parse(tr_anno, target_size = (224, 224))
te_x, te_mask = parse(te_anno, target_size = (224, 224))

max_cnt = 0
for mask in tr_mask + te_mask:
    if max_cnt < len(mask):
        max_cnt = len(mask)

tr_x = np.array(tr_x, dtype = np.float32) * 1/255
tr_mask = [np.pad(mask, [[0, max_cnt - len(mask)], [0, 0], [0, 0]]) for mask in tr_mask]
tr_mask = np.expand_dims(np.array(tr_mask, dtype = np.float32), axis = -1)

te_x = np.array(te_x, dtype = np.float32) * 1/255
te_mask = [np.pad(mask, [[0, max_cnt - len(mask)], [0, 0], [0, 0]]) for mask in te_mask]
te_mask = np.expand_dims(np.array(te_mask, dtype = np.float32), axis = -1)

tr_mask = np.max(tr_mask, axis = 1)
te_mask = np.max(te_mask, axis = 1)

batch_size = 2

tr_data = pipe((tr_x, tr_mask), batch_size = batch_size, shuffle = True)
te_data = pipe((te_x, te_mask), batch_size = batch_size, shuffle = False)

In [ ]:
import hrnet

"""
#hrnet v2 with backbone (custom development)
x = tf.keras.layers.Input(shape = (224, 224, 3))
model = tf.keras.applications.MobileNetV2(input_tensor = x, include_top = False)
layers = ["block_3_expand_relu", "block_6_expand_relu", "block_13_expand_relu", "out_relu"]
layers = [model.get_layer(l).output for l in layers]
out = hrnet.hrnet18_v2(layers, n_class = 2, include_top = True, mode = "ocr")
out = tf.keras.layers.UpSampling2D((4, 4))(out)
model = tf.keras.Model(x, out)

#manual initailize (hrnet18_v2 + ocr)
x = tf.keras.layers.Input(shape = (224, 224, 3))
out = hrnet_v2(x, n_class = 2, include_top = True, n_channel = [18, 36, 72, 144], n_module = [1, 4, 3], n_branch = [2, 3, 4], stage1_channel = 64, stage1_module = 1, ocr_feature_channel = 512, ocr_attention_channel = 256, ocr_dropout_rate = 0.05, ocr_scale = 1, mode = "ocr")
out = tf.keras.layers.UpSampling2D((4, 4))(out)
model = tf.keras.Model(x, out)
"""
#hrnet v2
x = tf.keras.layers.Input(shape = (224, 224, 3))
out = hrnet.hrnet18_v2(x, n_class = 2, include_top = True, mode = "ocr") #mode = seg > hrnet v2 + semantic segmentation, clsf > hrnet v2 + classifier, ocr > hrnet v2 + ocr + semantic segmentation
out = tf.keras.layers.UpSampling2D((4, 4))(out)
model = tf.keras.Model(x, out)

In [ ]:
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.SGD(1e-3, momentum = 0.9, nesterov = True)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, optimizer = opt, metrics = metric)

In [ ]:
model.fit(tr_data, validation_data = te_data, epochs = 3)

Epoch 1/3
31/31 [==============================] - 34s 288ms/step - loss: 1.3307 - sparse_categorical_accuracy: 0.7193 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.0794
Epoch 2/3
31/31 [==============================] - 4s 116ms/step - loss: 0.6740 - sparse_categorical_accuracy: 0.4599 - val_loss: 0.6833 - val_sparse_categorical_accuracy: 0.2136
Epoch 3/3
31/31 [==============================] - 4s 129ms/step - loss: 0.6726 - sparse_categorical_accuracy: 0.4169 - val_loss: 0.6909 - val_sparse_categorical_accuracy: 0.3237


In [ ]:
with open("model.json", mode = "w") as file:
    file.write(model.to_json())
model.save_weights("model.h5")

In [ ]:
with open("model.json", mode = "r") as file:
    model = tf.keras.models.model_from_json(file.read())
model.load_weights("model.h5")

In [ ]:
loss = tf.keras.losses.sparse_categorical_crossentropy
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, metrics = metric)
model.evaluate(te_data)

7/7 [==============================] - 6s 48ms/step - loss: 0.6909 - sparse_categorical_accuracy: 0.3237


[0.6908893585205078, 0.32372358441352844]